In [ ]:
dataset_name = "Departures"
pdf_path = "data/raw/MadridBarajasDeparturesShort.pdf"

In [10]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# PDF processing
import pdfplumber

# Regular expressions for text processing
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time handling
from datetime import datetime, timedelta

In [11]:
def clean_time(time_str):
    """Clean corrupted time strings from PDF extraction"""
    # Extract only digits and colon
    digits_only = ''.join(char for char in time_str if char.isdigit() or char == ':')
    
    # Try to find a time pattern (HH:MM)
    time_match = re.search(r'(\d{1,2}):(\d{2})', digits_only)
    
    if time_match:
        hours, minutes = map(int, time_match.groups())
        # Validate hours and minutes
        if 0 <= hours <= 23 and 0 <= minutes <= 59:
            return f"{hours:02d}:{minutes:02d}"
    return None

def clean_airline_name(parts):
    """Clean airline name by removing registration numbers and status tags"""
    # Pattern for registration numbers and other tags in parentheses
    tag_pattern = r'\([^)]+\)'
    # Join parts and remove any parenthetical content
    airline_name = ' '.join(parts)
    return re.sub(tag_pattern, '', airline_name).strip()

import sqlite3
from datetime import datetime

def extract_flights_from_pdf(pdf_path):
    """Extract flight information from PDF file and return as DataFrame"""
    flights = []
    in_target_section = False
    
    print("Opening PDF file...")
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            lines = page.extract_text().split('\n')
            
            for line in lines:
                # Check for date headers
                if "Thursday" in line and "Mar 13" in line:
                    in_target_section = True
                    print("Found target date section")
                    continue
                elif "day" in line and "Mar" in line and in_target_section:
                    print("Reached end of target date section")
                    in_target_section = False
                    continue
                
                # Only process lines in target section
                if in_target_section:
                    try:
                        parts = line.split()
                        if len(parts) < 5:  # Need minimum parts for valid entry
                            continue
                        
                        # Extract and validate time
                        time = clean_time(parts[0])
                        if not time:
                            continue
                        
                        # Extract flight number
                        flight = parts[1]
                        
                        # Find airport code position
                        origin_end = -1
                        for i, part in enumerate(parts[2:], 2):
                            if re.match(r'.*\([A-Z]{3}\)$', part):
                                origin_end = i
                                break
                        
                        if origin_end == -1:  # No valid airport code found
                            continue
                        
                        # Extract origin with airport code
                        origin = ' '.join(parts[2:origin_end + 1])
                        
                        # Process remaining parts for airline and aircraft
                        remaining = parts[origin_end + 1:]
                        
                        # Split remaining parts into airline and aircraft
                        aircraft_found = False
                        aircraft_type = None
                        airline_parts = []
                        
                        for part in remaining:
                            # Skip registration numbers and other parenthetical content
                            if re.match(r'\([^)]+\)', part):
                                continue
                            # Identify aircraft type
                            elif not aircraft_found and re.match(r'^[A-Z]\d+|[A-Z]{3,4}$', part):
                                aircraft_type = part
                                aircraft_found = True
                            # Add to airline name if not aircraft type
                            elif not aircraft_found:
                                airline_parts.append(part)
                        
                        if aircraft_type and airline_parts:
                            flight_data = {
                                "time": time,
                                "flight": flight,
                                "external": origin,
                                "airline": ' '.join(airline_parts).strip(),
                                "aircraft": aircraft_type,
                                "date": "2024-03-13"
                            }
                            flights.append(flight_data)
                            
                    except Exception as e:
                        print(f"Error processing line: {line}")
                        print(f"Error details: {str(e)}")
                        continue

    # Create DataFrame
    df = pd.DataFrame(flights)
    if len(df) > 0:
        print(f"\nTotal flights found: {len(df)}")
        print("\nSample of extracted data:")
        print(df.head())
    else:
        print("No valid flights found")
    
    return df

In [12]:
schengen_airports = {
    # Austria
    'VIE': 'Austria', 'GRZ': 'Austria', 'INN': 'Austria', 'SZG': 'Austria', 'KLU': 'Austria', 'LNZ': 'Austria',

    # Belgium
    'BRU': 'Belgium', 'CRL': 'Belgium', 'ANR': 'Belgium', 'LGG': 'Belgium', 'OST': 'Belgium',

    # Czech Republic
    'PRG': 'Czech Republic', 'BRQ': 'Czech Republic', 'OSR': 'Czech Republic', 'KLV': 'Czech Republic', 'PED': 'Czech Republic',

    # Denmark
    'CPH': 'Denmark', 'BLL': 'Denmark', 'AAL': 'Denmark', 'AAR': 'Denmark', 'EBJ': 'Denmark',

    # Estonia
    'TLL': 'Estonia', 'TAY': 'Estonia', 'EPU': 'Estonia', 'URE': 'Estonia',

    # Finland
    'HEL': 'Finland', 'TMP': 'Finland', 'TKU': 'Finland', 'OUL': 'Finland', 'RVN': 'Finland',

    # France
    'CDG': 'France', 'ORY': 'France', 'NCE': 'France', 'LYS': 'France', 'MRS': 'France', 'BOD': 'France', 'TLS': 'France', 'NTE': 'France',

    # Germany
    'FRA': 'Germany', 'MUC': 'Germany', 'DUS': 'Germany', 'TXL': 'Germany', 'HAM': 'Germany', 'STR': 'Germany', 'CGN': 'Germany', 'LEJ': 'Germany',

    # Greece
    'ATH': 'Greece', 'SKG': 'Greece', 'HER': 'Greece', 'RHO': 'Greece', 'CHQ': 'Greece', 'JMK': 'Greece',

    # Hungary
    'BUD': 'Hungary', 'DEB': 'Hungary', 'SOB': 'Hungary', 'PEV': 'Hungary',

    # Iceland
    'KEF': 'Iceland', 'RKV': 'Iceland', 'AEY': 'Iceland', 'EGS': 'Iceland',

    # Italy
    'FCO': 'Italy', 'MXP': 'Italy', 'LIN': 'Italy', 'VCE': 'Italy', 'TRN': 'Italy', 'BLQ': 'Italy', 'NAP': 'Italy', 'PSA': 'Italy',

    # Latvia
    'RIX': 'Latvia', 'VNT': 'Latvia', 'LPX': 'Latvia', 'RGA': 'Latvia',

    # Lithuania
    'VNO': 'Lithuania', 'KUN': 'Lithuania', 'PLQ': 'Lithuania', 'SQQ': 'Lithuania',

    # Luxembourg
    'LUX': 'Luxembourg',

    # Malta
    'MLA': 'Malta', 'GOZ': 'Malta',

    # Netherlands
    'AMS': 'Netherlands', 'EIN': 'Netherlands', 'RTM': 'Netherlands', 'GRQ': 'Netherlands', 'MST': 'Netherlands',

    # Norway
    'OSL': 'Norway', 'BGO': 'Norway', 'SVG': 'Norway', 'TOS': 'Norway', 'TRD': 'Norway',

    # Poland
    'WAW': 'Poland', 'KRK': 'Poland', 'GDN': 'Poland', 'WRO': 'Poland', 'POZ': 'Poland', 'KTW': 'Poland',

    # Portugal
    'LIS': 'Portugal', 'OPO': 'Portugal', 'FAO': 'Portugal', 'PDL': 'Portugal', 'FNC': 'Portugal',

    # Slovakia
    'BTS': 'Slovakia', 'KSC': 'Slovakia', 'TAT': 'Slovakia', 'PZY': 'Slovakia',

    # Slovenia
    'LJU': 'Slovenia', 'MBX': 'Slovenia', 'POW': 'Slovenia',

    # Spain
    'MAD': 'Spain', 'BCN': 'Spain', 'AGP': 'Spain', 'PMI': 'Spain', 'VLC': 'Spain', 'SVQ': 'Spain', 'BIO': 'Spain', 'ALC': 'Spain',

    # Sweden
    'ARN': 'Sweden', 'GOT': 'Sweden', 'MMX': 'Sweden', 'LLA': 'Sweden', 'BMA': 'Sweden',

    # Switzerland
    'ZRH': 'Switzerland', 'GVA': 'Switzerland', 'BSL': 'Switzerland', 'LUG': 'Switzerland', 'BRN': 'Switzerland',

    # Liechtenstein (No airports, uses Switzerland's airports)
}

In [13]:
def store_schengen_flights(df, db_path=dataset_name + ".db"):
    """Store Schengen flights in SQLite database
    
    Args:
        df (pd.DataFrame): DataFrame containing flight data
        db_path (str): Path to SQLite database file
    """
    try:
        # Filter for Schengen flights
        schengen_flights = df[df['external'].apply(lambda x: 
            bool(re.search(r'\(([A-Z]{3})\)', x) and 
                 re.search(r'\(([A-Z]{3})\)', x).group(1) in schengen_airports)
        )].copy()
        
        print(f"\nTotal Schengen flights found: {len(schengen_flights)}")
        
        # Connect to database
        conn = sqlite3.connect(db_path)
        
        # Create table if it doesn't exist
        conn.execute('''
            CREATE TABLE IF NOT EXISTS flights (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                time TEXT,
                flight TEXT,
                external TEXT,
                airline TEXT,
                aircraft TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Insert Schengen flights
        schengen_flights.to_sql('flights', conn, if_exists='append', index=False)
        print(f"\nSchengen flights successfully stored in {db_path}")
        
        # Show database statistics
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM flights")
        total_flights = cursor.fetchone()[0]
        print(f"Total flights in database: {total_flights}")
        
        conn.close()
        
        return schengen_flights
        
    except Exception as e:
        print(f"Error storing data in database: {str(e)}")
        return None